In [15]:
from llama_index.core import PromptTemplate
import json
import torch
from llama_index.core import Document
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core import StorageContext
from llama_index.core import VectorStoreIndex
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.vector_stores.weaviate import WeaviateVectorStore
from pyvi import ViTokenizer
import weaviate
from weaviate.classes.init import Auth

In [16]:

DATA_COLLECTION = "ND168"
DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"
MODEL_NAME = "dangvantuan/vietnamese-document-embedding"
CHUNK_SIZE = 512  # Optimized for Vietnamese text
CHUNK_OVERLAP = 50  # Small overlap to maintain context

In [17]:
# Setup vector store
# client = weaviate.connect_to_weaviate_cloud(
#     cluster_url=WEAVIATE_URL,
#     auth_credentials=Auth.api_key(WEAVIATE_API_KEY),
# )


client = weaviate.connect_to_local(
                     host="192.168.100.125",
                        port=8080,
                        grpc_port=50051
                )
vector_store = WeaviateVectorStore(
    weaviate_client=client,
    index_name=DATA_COLLECTION
)

In [18]:
index = VectorStoreIndex.from_vector_store(
    vector_store=vector_store,
    embed_model = HuggingFaceEmbedding(model_name=MODEL_NAME, device=DEVICE, trust_remote_code=True,cache_folder="/home/drissdo/.cache/huggingface/hub"),
)

In [19]:
# Create retriever
retriever = index.as_retriever(
    vector_store_query_mode="hybrid",
    similarity_top_k=20,
    alpha=0.3,
)

/home/drissdo/Desktop/VN_LAW_QA_SYSTEM/venv/lib/python3.11/site-packages/weaviate/warnings.py:314: ResourceWarning: Con004: The connection to Weaviate was not closed properly. This can lead to memory leaks.
            Please make sure to close the connection using `client.close()`.
  warnings.warn(


In [20]:
question = "Đối với phương tiện, vi phạm đỗ xe trên vỉa hè sẽ bị xử phạt như thế nào?"
retrieved_docs = retriever.retrieve(question)

for i, node in enumerate(retrieved_docs):
    print(f"Result {i+1}:\n")
    print(f"Text: {node.text}\n")
    print(f"Metadata: {node.metadata}\n")
    print(f"Score: {node.score}\n")
    print("="*50)

Result 1:

Text: loại phương_tiện : ô_tô mức phạt : phạt tiền từ 800.000 đồng đến 1.000.000 đồng_nội_dung vi_phạm : đỗ xe không sát theo lề_đường , vỉa_hè phía bên phải theo chiều đi ( ô_tô )

Metadata: {'category': 'ô tô', 'fine_amount': 'Phạt tiền từ 800.000 đồng đến 1.000.000 đồng', 'violation_type': 'khác', 'original_text': 'Loại phương tiện: ô tô\nMức phạt: Phạt tiền từ 800.000 đồng đến 1.000.000 đồng\nNội dung vi phạm: Đỗ xe không sát theo lề đường, vỉa hè phía bên phải theo chiều đi (ô tô)'}

Score: 0.8395899534225464

Result 2:

Text: loại phương_tiện : ô_tô mức phạt : phạt tiền từ 6.000.000 đồng đến 8.000.000 đồng_nội_dung vi_phạm : điều_khiển xe đi trên vỉa_hè , trừ trường_hợp điều_khiển xe đi qua vỉa_hè để vào nhà , cơ_quan ( ô_tô )

Metadata: {'category': 'ô tô', 'fine_amount': 'Phạt tiền từ 6.000.000 đồng đến 8.000.000 đồng', 'violation_type': 'khác', 'original_text': 'Loại phương tiện: ô tô\nMức phạt: Phạt tiền từ 6.000.000 đồng đến 8.000.000 đồng\nNội dung vi phạm: Điều 